In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import folium.plugins
from folium import Figure
from folium.plugins import HeatMapWithTime

# For real distance
import osmnx as ox
import networkx as nx

import src.cleaning as cleaning
import src.visualizations as vis

In [5]:
# Sample to test functions:
apr_14 = pd.read_csv('../../../Desktop/final_project_data/2014/2014-04 - Citi Bike trip data.csv')

# 1. CLEANING PROCESS

# Rename columns

In [6]:
apr_14 = cleaning.rename_columns(apr_14)

#  Straight line Distance

In [7]:
apr_14 = cleaning.trip_distance(apr_14)

# Hour: start_hour & end_hour (same function, 2 columns)

- Integer value for the hour, e.g. 12, 18.

In [8]:
apr_14 = cleaning.get_hour (apr_14)

# Date: trip_date
- time series date format for date, including year, month and day, and excluding hour, minute and second.

In [9]:
apr_14 = cleaning.get_date (apr_14)

# Formatting: started_at and ended_at in datetime format

In [10]:
apr_14 = cleaning.datetime_format (apr_14)

#  Month, weekday, weekend

In [11]:
apr_14 = cleaning.get_categorical_date (apr_14)

# Cleaned and enriched dataframe to CSV

In [12]:
apr_14.to_csv('data/april_2014.csv', index=False)

In [13]:
april_14 = pd.read_csv('data/april_2014.csv')

# Open Street Map: real_distance
### Function to create subdataframes and save them as csv.

In [14]:
# in Collab

# cleaning.dataframe_split (df, n): Splits the dataframe in n subdataframes to make it more processable.
# cleaning.get_real_distance (df): uses osmnx and networkx to compute the shortest available path distance.

# rideable_type

In [15]:
# Infer from 2022 datasets.

# trip_cost ()

¡You need to get bike type before!

- float $ value in function of rideable_type, duration and member_casual.

Subscriber
- Classic.
If trip_duration < 45*60, cost = 0
Elif trip_duration > 45*60, cost = (trip_duration - 45 * 60) / 60 * 0.17

- Electric
If trip_duration =< 45*60: cost = trip_duration / 60 * 0.17, limit 3.
If trip_duration > 45*60: cost = 3 + (trip_duration - 45 * 60) / 60 * 0.17

Casual
- Single trip 
    - Casual & - Electric
    If trip_duration <= 30*60 -> 4.49
    Elif trip_duration > 30 * 60 -> 4.49 + (trip_duration - 30 * 60) / 60 * 0.26

- Day Pass
    - Casual 
    - Electric

In [16]:
# need to know rideable type.

# Station non-bike trips balance

## Bike Route (not by human usage):

All bikes journey function includes bike route function.

In [17]:
all_bikes_journey_list = cleaning.all_bikes_journey (april_14)

## Non user trip mobility dictionary and information:

### Three functions that enrich a dictionary containing all info regarding mobility from one station to another through trucks.

In [18]:
stations_transfers_dictionary = cleaning.non_trip_mobility_dict (apr_14)
stations_transfers_dictionary = cleaning.transportations (stations_transfers_dictionary, all_bikes_journey_list)
stations_transfers_dictionary = cleaning.station_balance (stations_transfers_dictionary)

# Truck trips

Record all bike movements not attributable to user trips.
- Bike ID
- Date range
- Transported from
- Transported to

Sort by date, group by station.

### All transfers use single bike truck transfers function:

In [20]:
total_transfers = cleaning.all_transfers (april_14)

In [23]:
total_transfers = cleaning.datetime_format_trucks (total_transfers)

In [27]:
total_transfers.to_csv('data/truck_transfers.csv', index=False)

# 2. Visualizations

In [30]:
# Functions defined in source folder, deployed in streamlit app.

In [1]:
# Hourly distribution by day:
def hourly_dist (df, day):
    test_map_viz = df[['start_hour', 'started_at', 'bike_id', 'start_lat', 'start_lng', 'weekday']]
    test_map_viz = test_map_viz[test_map_viz['weekday'] == day]

    lat_lng_list = []
    for i in range(24):
        temp=[]
        for index, row in test_map_viz[test_map_viz['start_hour'] == i].iterrows():
            temp.append([row['start_lat'],row['start_lng']])
        lat_lng_list.append(temp)

    figure1 = Figure(width=850,height=550)
    new_york1 = folium.Map(location=[40.712776, -74.005974],zoom_start=12)

    figure1.add_child(new_york1)
    folium.TileLayer('cartodbpositron').add_to(new_york1)
    gradient = {.33: 'white', .66: 'lightblue', 1: 'blue'}

    HeatMapWithTime(lat_lng_list, radius=5, auto_play=True, position='bottomright', gradient=gradient).add_to(new_york1)

    return figure1

In [ ]:
hourly_dist (april_14, 'Monday')

# 3. Your Trip

In [2]:
trucks = pd.read_csv('data/truck_transfers.csv')
april = pd.read_csv('data/april_2014.csv')

In [3]:
ALL_TRIPS = cleaning.concat_all_bike_trips (april)

/Users/gerardrius/Ironhack/projects/Final/src/cleaning.py:452: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  result_df = pd.DataFrame({'last_end': last_end, 'ended_at': etime, 'next_start': next_start, 'started_at': stime})
/Users/gerardrius/Ironhack/projects/Final/src/cleaning.py:452: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  result_df = pd.DataFrame({'last_end': last_end, 'ended_at': etime, 'next_start': next_start, 'started_at': stime})
/Users/gerardrius/Ironhack/projects/Final/src/cleaning.py:452: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series

In [4]:
ALL_TRIPS.to_csv('data/all_trips.csv', index = False)

In [5]:
a=pd.read_csv('data/all_trips.csv')

In [7]:
a.iloc[:500]

,last_end,ended_at,next_start,started_at,time_difference,bike_id
0,2008.0,2014-04-01 00:09:25.000,2008.0,2014-04-01 07:41:30.000,0 days 07:32:05,21062.0
1,224.0,2014-04-01 07:48:08.000,224.0,2014-04-01 08:54:32.000,0 days 01:06:24,21062.0
2,360.0,2014-04-01 08:58:20.000,360.0,2014-04-01 15:03:40.800,0 days 06:05:20.800000,21062.0
3,306.0,2014-04-02 09:19:43.200,306.0,2014-04-02 15:25:04.000,0 days 06:05:20.800000,21062.0
4,147.0,2014-04-02 15:32:24.000,147.0,2014-04-02 17:22:38.000,0 days 01:50:14,21062.0
...,...,...,...,...,...,...
495,151.0,2014-04-10 18:14:27.000,151.0,2014-04-10 18:14:53.000,0 days 00:00:26,20914.0
496,531.0,2014-04-10 18:20:36.000,531.0,2014-04-10 18:59:16.000,0 days 00:38:40,20914.0
497,445.0,2014-04-10 19:07:10.000,445.0,2014-04-10 21:20:33.800,0 days 02:13:23.800000,20914.0
498,519.0,2014-04-11 04:00:45.200,519.0,2014-04-11 06:14:09.000,0 days 02:13:23.800000,20914.0


In [31]:
import datetime

In [34]:
ALL_TRIPS.last_end.value_counts() == ALL_TRIPS.next_start.value_counts()

519.0     True
521.0     True
497.0     True
293.0     True
435.0     True
          ... 
119.0     True
372.0     True
431.0     True
2005.0    True
321.0     True
Length: 330, dtype: bool

In [55]:
date_range = []
for i in range(1,10):
    date_range.append(f'2014-04-0{i}')

for i in range(10,31):
    date_range.append(f'2014-04-{i}')

In [62]:
def station_max_min_differences (df, times):
    # time to datetime format
    differences = []

    for date in times:
        date_time_obj = datetime.datetime.strptime(date, '%Y-%m-%d')

        ends = df[df['ended_at'] <= date_time_obj].last_end.value_counts().to_frame().reset_index().rename(columns = {'index': 'end_id', 'last_end': 'counts'})
        starts = df[df['started_at'] <= date_time_obj].next_start.value_counts().to_frame().reset_index().rename(columns = {'index': 'start_id', 'next_start': 'counts'})

        try:
            differences.append(ends.iloc[0]['counts'] - starts.iloc[0]['counts'])
        except:
            differences.append(0)

    return differences

In [63]:
station_max_min_differences (ALL_TRIPS, date_range)

[0,
 3.0,
 5.0,
 11.0,
 27.0,
 13.0,
 14.0,
 24.0,
 20.0,
 15.0,
 11.0,
 18.0,
 21.0,
 14.0,
 18.0,
 33.0,
 23.0,
 38.0,
 99.0,
 90.0,
 118.0,
 26.0,
 28.0,
 2.0,
 15.0,
 30.0,
 10.0,
 12.0,
 5.0,
 16.0]

In [50]:
def station_max_min_differences (df, time_range):
    # time to datetime format
    time_range_datetime = []
    for date in time_range:
        date_time_obj = datetime.datetime.strptime(date, '%Y-%m-%d')
        time_range_datetime.append(date_time_obj)

    differences = []

    for date in time_range_datetime:
        ends = df[df['ended_at'] <= date].last_end.value_counts().to_frame().reset_index().rename(columns = {'index': 'end_id', 'last_end': 'counts'})
        starts = df[df['started_at'] <= date].next_start.value_counts().to_frame().reset_index().rename(columns = {'index': 'start_id', 'next_start': 'counts'})

        differences.append(ends.iloc[0]['counts'] - starts.iloc[0]['counts'])

    return differences